In [1]:
#importing all libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import json
import time
from datetime import datetime
from pandas.io.json import json_normalize
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import seaborn as sns
import gc
gc.enable()

In [34]:
#reading training data in chunk
import pandas as pd
csv_path = file = "/home/rk9cx/train_v2.csv"
chunksize = 100000
features = ['channelGrouping', 'date', 'fullVisitorId', 'visitId',
                'visitNumber', 'visitStartTime', 'device_browser',
                'device_deviceCategory', 'device_isMobile', 'device_operatingSystem',
                'geoNetwork_city', 'geoNetwork_continent', 'geoNetwork_country',
                'geoNetwork_metro', 'geoNetwork_networkDomain', 'geoNetwork_region',
                'geoNetwork_subContinent', 'totals_bounces', 'totals_hits',
                'totals_newVisits', 'totals_pageviews', 'totals_transactionRevenue',
                'trafficSource_adContent', 'trafficSource_campaign',
                'trafficSource_isTrueDirect', 'trafficSource_keyword',
                'trafficSource_medium', 'trafficSource_referralPath',
                'trafficSource_source']
JSON_COLS = ['device', 'geoNetwork', 'totals', 'trafficSource']
print('Load {}'.format(csv_path))
df_reader = pd.read_csv(csv_path, converters={ column: json.loads for column in JSON_COLS },
                            dtype={ 'date': str, 'fullVisitorId': str, 'sessionId': str },
                            chunksize=chunksize)
res = pd.DataFrame()
for cidx, df in enumerate(df_reader):
    df.reset_index(drop=True, inplace=True)
    for col in JSON_COLS:
        col_as_df = json_normalize(df[col])
        col_as_df.columns = ['{}_{}'.format(col, subcol) for subcol in col_as_df.columns]
        df = df.drop(col, axis=1).merge(col_as_df, right_index=True, left_index=True)
    res = pd.concat([res, df[features]], axis=0).reset_index(drop=True)
    del df
    gc.collect()
    print('{}: {}'.format(cidx + 1, res.shape))


Load /home/rk9cx/train_v2.csv
1: (100000, 29)
2: (200000, 29)
3: (300000, 29)
4: (400000, 29)
5: (500000, 29)
6: (600000, 29)
7: (700000, 29)
8: (800000, 29)
9: (900000, 29)
10: (1000000, 29)
11: (1100000, 29)
12: (1200000, 29)
13: (1300000, 29)
14: (1400000, 29)
15: (1500000, 29)
16: (1600000, 29)
17: (1700000, 29)
18: (1708337, 29)


In [35]:
train_df=res
train_df.head()

,channelGrouping,date,fullVisitorId,visitId,visitNumber,visitStartTime,device_browser,device_deviceCategory,device_isMobile,device_operatingSystem,...,totals_newVisits,totals_pageviews,totals_transactionRevenue,trafficSource_adContent,trafficSource_campaign,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source
0,Organic Search,20171016,3162355547410993243,1508198450,1,1508198450,Firefox,desktop,False,Windows,...,1,1,NaN,NaN,(not set),NaN,water bottle,organic,NaN,google
1,Referral,20171016,8934116514970143966,1508176307,6,1508176307,Chrome,desktop,False,Chrome OS,...,NaN,2,NaN,NaN,(not set),NaN,NaN,referral,/a/google.com/transportation/mtv-services/bike...,sites.google.com
2,Direct,20171016,7992466427990357681,1508201613,1,1508201613,Chrome,mobile,True,Android,...,1,2,NaN,NaN,(not set),True,NaN,(none),NaN,(direct)
3,Organic Search,20171016,9075655783635761930,1508169851,1,1508169851,Chrome,desktop,False,Windows,...,1,2,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
4,Organic Search,20171016,6960673291025684308,1508190552,1,1508190552,Chrome,desktop,False,Windows,...,1,2,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google


In [9]:
#reading testing data in chunks
import pandas as pd
csv_path = file = "/home/rk9cx/test_v2.csv"
chunksize = 50000
features = ['channelGrouping', 'date', 'fullVisitorId', 'visitId',
                'visitNumber', 'visitStartTime', 'device_browser',
                'device_deviceCategory', 'device_isMobile', 'device_operatingSystem',
                'geoNetwork_city', 'geoNetwork_continent', 'geoNetwork_country',
                'geoNetwork_metro', 'geoNetwork_networkDomain', 'geoNetwork_region',
                'geoNetwork_subContinent', 'totals_bounces', 'totals_hits',
                'totals_newVisits', 'totals_pageviews', 'totals_transactionRevenue',
                'trafficSource_adContent', 'trafficSource_campaign',
                'trafficSource_isTrueDirect', 'trafficSource_keyword',
                'trafficSource_medium', 'trafficSource_referralPath',
                'trafficSource_source']
JSON_COLS = ['device', 'geoNetwork', 'totals', 'trafficSource']
print('Load {}'.format(csv_path))
df_reader = pd.read_csv(csv_path, converters={ column: json.loads for column in JSON_COLS },
                            dtype={ 'date': str, 'fullVisitorId': str, 'sessionId': str },
                            chunksize=chunksize)
res = pd.DataFrame()
for cidx, df in enumerate(df_reader):
    df.reset_index(drop=True, inplace=True)
    for col in JSON_COLS:
        col_as_df = json_normalize(df[col])
        col_as_df.columns = ['{}_{}'.format(col, subcol) for subcol in col_as_df.columns]
        df = df.drop(col, axis=1).merge(col_as_df, right_index=True, left_index=True)
    res = pd.concat([res, df[features]], axis=0).reset_index(drop=True)
    del df
    gc.collect()
    print('{}: {}'.format(cidx + 1, res.shape))

Load /home/rk9cx/test_v2.csv
1: (50000, 29)
2: (100000, 29)
3: (150000, 29)
4: (200000, 29)
5: (250000, 29)
6: (300000, 29)
7: (350000, 29)
8: (400000, 29)
9: (401589, 29)


In [11]:
test_df = res
test_df.head()

,channelGrouping,date,fullVisitorId,visitId,visitNumber,visitStartTime,device_browser,device_deviceCategory,device_isMobile,device_operatingSystem,...,totals_newVisits,totals_pageviews,totals_transactionRevenue,trafficSource_adContent,trafficSource_campaign,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source
0,Organic Search,20180511,7460955084541987166,1526099341,2,1526099341,Chrome,mobile,True,Android,...,NaN,3,NaN,(not set),(not set),True,(not provided),organic,(not set),google
1,Direct,20180511,460252456180441002,1526064483,166,1526064483,Chrome,desktop,False,Macintosh,...,NaN,3,NaN,(not set),(not set),True,(not set),(none),(not set),(direct)
2,Organic Search,20180511,3461808543879602873,1526067157,2,1526067157,Chrome,desktop,False,Chrome OS,...,NaN,3,NaN,(not set),(not set),True,(not provided),organic,(not set),google
3,Direct,20180511,975129477712150630,1526107551,4,1526107551,Chrome,mobile,True,iOS,...,NaN,4,NaN,(not set),(not set),True,(not set),(none),(not set),(direct)
4,Organic Search,20180511,8381672768065729990,1526060254,1,1526060254,Internet Explorer,tablet,True,Windows,...,1,4,NaN,(not set),(not set),NaN,(not provided),organic,(not set),google


In [36]:
#treating date in Train    
train_df['date'] = pd.to_datetime(train_df['visitStartTime'], unit='s')
train_df['dayofweek'] = train_df['date'].dt.dayofweek
train_df['month'] = train_df['date'].dt.month
train_df['day'] = train_df['date'].dt.day
train_df['hour'] = train_df['date'].dt.hour

In [13]:
#treating date in test    
test_df['date'] = pd.to_datetime(test_df['visitStartTime'], unit='s')
test_df['dayofweek'] = test_df['date'].dt.dayofweek
test_df['month'] = test_df['date'].dt.month
test_df['day'] = test_df['date'].dt.day
test_df['hour'] = test_df['date'].dt.hour

In [37]:
#converting to float
for col in ['visitNumber', 'totals_hits', 'totals_pageviews']:
    train_df[col] = train_df[col].astype(float)
    
#filling empty values in Train
train_df['trafficSource_isTrueDirect'].fillna(False, inplace=True)

In [15]:
#converting to float
for col in ['visitNumber', 'totals_hits', 'totals_pageviews']:
    test_df[col] = test_df[col].astype(float)
    
#filling empty values in test
test_df['trafficSource_isTrueDirect'].fillna(False, inplace=True)

In [38]:
#processing Devices
train_df['browser_category'] = train_df['device_browser'] + '_' + train_df['device_deviceCategory']
train_df['browser_operatingSystem'] = train_df['device_browser'] + '_' + train_df['device_operatingSystem']
train_df['source_country'] = train_df['trafficSource_source'] + '_' + train_df['geoNetwork_country']


In [ ]:
test_df['browser_category'] = test_df['device_browser'] + '_' + test_df['device_deviceCategory']
test_df['browser_operatingSystem'] = test_df['device_browser'] + '_' + test_df['device_operatingSystem']
test_df['source_country'] = test_df['trafficSource_source'] + '_' + test_df['geoNetwork_country']



In [39]:
#processing geonetwork
train_df['count_hits_nw_domain'] = train_df.groupby('geoNetwork_networkDomain')['totals_hits'].transform('count')
train_df['sum_hits_nw_domain'] = train_df.groupby('geoNetwork_networkDomain')['totals_hits'].transform('sum')
train_df['count_pvs_nw_domain'] = train_df.groupby('geoNetwork_networkDomain')['totals_pageviews'].transform('count')
train_df['sum_pvs_nw_domain'] = train_df.groupby('geoNetwork_networkDomain')['totals_pageviews'].transform('sum')


In [ ]:
test_df['count_hits_nw_domain'] = test_df.groupby('geoNetwork_networkDomain')['totals_hits'].transform('count')
test_df['sum_hits_nw_domain'] = test_df.groupby('geoNetwork_networkDomain')['totals_hits'].transform('sum')
test_df['count_pvs_nw_domain'] = test_df.groupby('geoNetwork_networkDomain')['totals_pageviews'].transform('count')
test_df['sum_pvs_nw_domain'] = test_df.groupby('geoNetwork_networkDomain')['totals_pageviews'].transform('sum')

In [40]:
# Extract features.
excluded_cols = ['date', 'fullVisitorId', 'visitId', 'visitStartTime', 'totals_transactionRevenue']
objt_cols = [col for col in train_df.columns if col not in excluded_cols and train_df[col].dtypes == object]
for col in objt_cols:
    train_df[col], indexer = pd.factorize(train_df[col])
    test_df[col] = indexer.get_indexer(test_df[col])
bool_cols = [col for col in train_df.columns if col not in excluded_cols and train_df[col].dtypes == bool]
for col in bool_cols:
    train_df[col] = train_df[col].astype(int)
    test_df[col] = test_df[col].astype(int)
# Fill NaN
numb_cols = [col for col in train_df.columns if col not in excluded_cols and col not in objt_cols]
for col in numb_cols:       
    train_df[col] = train_df[col].fillna(0)
    test_df[col] = test_df[col].fillna(0)

In [41]:
#Dropping columns that do repeat and have no unique values
cols_to_drop = [col for col in train_df.columns if train_df[col].nunique(dropna=False) == 1]
train_df.drop(cols_to_drop, axis=1, inplace=True)
test_df.drop([col for col in cols_to_drop if col in test_df.columns], axis=1, inplace=True)

In [42]:
#final processing step
EXCLUDED_COLS = ['date', 'fullVisitorId', 'visitId', 'visitStartTime', 'totals_transactionRevenue']
test_fvid = test_df[['fullVisitorId']].copy()
target_values = np.log1p(train_df['totals_transactionRevenue'].fillna(0).astype(float))
train_df.drop(EXCLUDED_COLS, axis=1, inplace=True)
test_df.drop(EXCLUDED_COLS, axis=1, inplace=True)

In [33]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=123456)

In [44]:
train_tvals = target_values

In [47]:
#using decision trees
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor as dtr
# try fitting a decision tree regression model...
DTR_1 = dtr(max_depth=None) # declare the regression model form. Let the depth be default.

In [48]:
scores_dtr = cross_val_score(DTR_1, train_df, train_tvals, cv=10,scoring='explained_variance') # 10-fold cross validation
print('scores for k=10 fold validation:',scores_dtr)

scores for k=10 fold validation: [-0.45098684 -0.37731939 -0.53416407 -0.44969335 -0.39463394 -0.49130642
 -0.43176019 -0.43254027 -0.49386857 -0.48159744]


In [49]:
#found explained variance as a performance score - https://scikit-learn.org/stable/modules/model_evaluation.html#explained-variance-score
print("Est. explained variance: %0.2f (+/- %0.2f)" % (scores_dtr.mean(), scores_dtr.std() * 2))
#values closer to one are better and negaitve values indicate that the predictions are very far from the actual values

Est. explained variance: -0.45 (+/- 0.09)


In [52]:
train_df.to_csv('train.csv', index=False)
test_df.to_csv('test.csv', index=False)
train_tvals.to_csv('train_tvals.csv', index=False)
test_fvid.to_csv('test_fvid.csv', index=False)

In [2]:
#reading data into local machine
import os 
import pandas as pd
os.chdir("/home/rk9cx/")
test_df = pd.read_csv("test.csv", index_col=False)
train_df = pd.read_csv("train.csv", index_col=False)
train_tvals = pd.read_csv("train_tvals.csv", index_col=False)
test_fvids = pd.read_csv("test_fvid.csv", index_col=False)

In [4]:
#fitting a Random Forest Model
import numpy as np
from sklearn.ensemble import RandomForestRegressor as rfr
np.random.seed(11111)
model = rfr(n_estimators=200,max_depth=None)
model.fit(train_df.iloc[0:1708336,],train_tvals)

/apps/software/standard/core/anaconda3/5.2.0/lib/python3.6/site-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [7]:
#making predictions
y_output = model.predict(test_df.fillna(0))

In [5]:
#function for aggregating by user IDs and making the submission file
def submit(predictions, submission, filename):
    submission.loc[:, 'PredictedLogRevenue'] = predictions
    submission['PredictedLogRevenue'] = submission['PredictedLogRevenue'].fillna(0.0).apply(lambda x : 0.0 if x < 0 else x)
    grouped_test = submission[['fullVisitorId', 'PredictedLogRevenue']].groupby('fullVisitorId').sum().reset_index()
    grouped_test['PredictedLogRevenue'] = np.log1p(grouped_test['PredictedLogRevenue'])
    grouped_test['PredictedLogRevenue'] = grouped_test['PredictedLogRevenue'].fillna(0.0)
    grouped_test.to_csv(filename, index=False)

In [ ]:
#final submissions
submit(y_output, test_fvids, 'submission.csv')

In [13]:
#cross validation 
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor 
cross_validate(RandomForestRegressor(),train_df.iloc[0:1708336,],train_tvals,cv=5,return_train_score=True)

/apps/software/standard/core/anaconda3/5.2.0/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/apps/software/standard/core/anaconda3/5.2.0/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/apps/software/standard/core/anaconda3/5.2.0/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/apps/software/standard/core/anaconda3/5

{'fit_time': array([81.41530752, 81.14794636, 80.79956746, 87.00817037, 82.18678546]),
 'score_time': array([0.3021071 , 0.30846167, 0.3019948 , 0.31497073, 0.31857705]),
 'test_score': array([ 0.04338066,  0.02234614,  0.0216028 ,  0.03451401, -0.00624913]),
 'train_score': array([0.82555191, 0.82582811, 0.82559297, 0.82540232, 0.8247746 ])}